# *Pre-processing and Feature Extraction*

In [1]:
from utility.preProcessor import *
from utility.featureExtractor import *

In [2]:
data_path = './dataset/chb-mit-scalp'
for folder in (os.listdir(data_path)):
    data_dir = data_path+"/"+folder
    summary_dir = data_dir+"/"+folder+"-summary.txt"
    try:
       print(summary_dir)
       summary_df = chb_mit_parse_summary(summary_dir)
    #    chb_mit_edf_to_csv(data_dir)
       chb_mit_add_class_to_csv(data_dir, summary_df)  
    except Exception as e:
        print(summary_dir+ " exception " +str(e))
        
  

./dataset/chb-mit-scalp/chb16/chb16-summary.txt
Processed Row 0: chb16_01.csv
Processed Row 1: chb16_02.csv
Processed Row 2: chb16_03.csv
Processed Row 3: chb16_04.csv
Processed Row 4: chb16_05.csv
Processed Row 5: chb16_06.csv
Processed Row 6: chb16_07.csv
Processed Row 7: chb16_08.csv
Processed Row 8: chb16_09.csv
Processed Row 9: chb16_10.csv
Processed Row 10: chb16_11.csv
Processed Row 11: chb16_12.csv
Processed Row 12: chb16_13.csv
Processed Row 13: chb16_14.csv
Processed Row 14: chb16_15.csv
Processed Row 15: chb16_16.csv
Processed Row 16: chb16_17.csv
Processed Row 17: chb16_18.csv
Processed Row 18: chb16_19.csv
./dataset/chb-mit-scalp/chb09/chb09-summary.txt
Processed Row 0: chb09_01.csv
Processed Row 1: chb09_02.csv
Processed Row 2: chb09_03.csv
Processed Row 3: chb09_04.csv
Processed Row 4: chb09_05.csv
Processed Row 5: chb09_06.csv
Processed Row 6: chb09_07.csv
Processed Row 7: chb09_08.csv
Processed Row 8: chb09_09.csv
Processed Row 9: chb09_10.csv
Processed Row 10: chb09_1

In [10]:
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

# Processing function with dependency preserved
def process_folder(data_path, folder):
    data_dir = os.path.join(data_path, folder)
    print(f"Processing: {data_dir}")
    
    # Perform dependent operations
    chb_mit_edf_to_csv(data_dir)
    summary_file = os.path.join(data_dir, f"{folder}-summary.txt")
    summary_data = chb_mit_parse_summary(summary_file)
    chb_mit_add_class_to_csv(data_dir, summary_data)


data_path = os.path.abspath('./dataset/chb-mit-scalp')
folders = os.listdir(data_path)

# Use ProcessPoolExecutor and track progress
with ProcessPoolExecutor() as executor:
    futures = {executor.submit(process_folder, data_path, folder): folder for folder in folders}

    # Display progress
    for future in tqdm(as_completed(futures), total=len(folders), desc="Processing Folders"):
        try:
            future.result()  # Raise any exceptions that occurred
        except Exception as e:
            print(f"Error processing {futures[future]}: {e}")


Processing Folders:   0%|          | 0/24 [00:00<?, ?it/s]

Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb09Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb16Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb08Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb20Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb05Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb21Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb07Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb11Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb22Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb23Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb03Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb10Processing: /home/khansun/repos/seizure-detection/dataset/chb-mit-scalp/chb1

Processing Folders:   4%|▍         | 1/24 [08:00<3:04:01, 480.04s/it]

file processed: chb05_08.edf frequency: 256.0
file processed: chb19_07.edf frequency: 256.0
file processed: chb21_28.edf frequency: 256.0
file processed: chb11_60.edf frequency: 256.0
file processed: chb20_68.edf frequency: 256.0
file processed: chb10_06.edf frequency: 256.0
file processed: chb13_18.edf frequency: 256.0
file processed: chb15_45.edf frequency: 256.0
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
file processed: chb22_19.edf frequency: 256.0
file processed: chb07_05.edf frequency: 256.0
file processed: chb14_25.edf frequency: 256.0
file processed: chb03_19.edf frequency: 256.0
file processed: chb16_07.edf frequency: 256.0
file processed: chb12_06.edf frequency: 256.0
error <class 'Exception'> 
file processed: chb18_08.edf frequency: 256.0
file processed: chb09_08.edf frequency: 256.0
file processed: chb04_07.edf frequency: 256.0
error <class 'Exception'> 
file processed: chb01_31.edf frequency: 256.0
error <cla

Processing Folders:   8%|▊         | 2/24 [08:22<1:17:12, 210.58s/it]

...

Processing Folders:  92%|█████████▏| 22/24 [19:12<01:25, 42.81s/it]

file processed: chb09_06.edf frequency: 256.0
file processed: chb04_38.edf frequency: 256.0
error <class 'Exception'> 
file processed: chb09_17.edf frequency: 256.0
error <class 'Exception'> 
file processed: chb09_19.edf frequency: 256.0
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 
error <class 'Exception'> 


Processing Folders:  96%|█████████▌| 23/24 [20:03<00:45, 45.40s/it]

file processed: chb04_31.edf frequency: 256.0
file processed: chb04_06.edf frequency: 256.0
file processed: chb04_22.edf frequency: 256.0
error <class 'Exception'> 
file processed: chb04_04.edf frequency: 256.0
file processed: chb04_12.edf frequency: 256.0
file processed: chb04_23.edf frequency: 256.0
file processed: chb04_35.edf frequency: 256.0
file processed: chb04_40.edf frequency: 256.0
file processed: chb04_34.edf frequency: 256.0
file processed: chb04_19.edf frequency: 256.0
file processed: chb04_43.edf frequency: 256.0
file processed: chb04_26.edf frequency: 256.0
file processed: chb04_25.edf frequency: 256.0
file processed: chb04_05.edf frequency: 256.0
file processed: chb04_18.edf frequency: 256.0
file processed: chb04_09.edf frequency: 256.0
file processed: chb04_11.edf frequency: 256.0
file processed: chb04_42.edf frequency: 256.0
file processed: chb04_14.edf frequency: 256.0
file processed: chb04_03.edf frequency: 256.0
file processed: chb04_10.edf frequency: 256.0
file pr

Processing Folders: 100%|██████████| 24/24 [34:28<00:00, 86.17s/it] 


Processed Row 0: chb01_01.csv
Processed Row 1: chb01_02.csv
Processed Row 2: chb01_03.csv
Processed Row 3: chb01_04.csv
Processed Row 4: chb01_05.csv
Processed Row 5: chb01_06.csv
Processed Row 6: chb01_07.csv
Processed Row 7: chb01_08.csv
Processed Row 8: chb01_09.csv
Processed Row 9: chb01_10.csv
Processed Row 10: chb01_11.csv
Processed Row 11: chb01_12.csv
Processed Row 12: chb01_13.csv
Processed Row 13: chb01_14.csv
Processed Row 14: chb01_15.csv
Processed Row 15: chb01_16.csv
Processed Row 16: chb01_17.csv
Processed Row 17: chb01_18.csv
Processed Row 18: chb01_19.csv
Processed Row 19: chb01_20.csv
Processed Row 20: chb01_21.csv
Processed Row 21: chb01_22.csv
Processed Row 22: chb01_23.csv
Processed Row 23: chb01_24.csv
Processed Row 24: chb01_25.csv
Processed Row 25: chb01_26.csv
Processed Row 26: chb01_27.csv
Processed Row 27: chb01_29.csv
Processed Row 28: chb01_30.csv
Processed Row 29: chb01_31.csv
Processed Row 30: chb01_32.csv
Processed Row 31: chb01_33.csv
Processed Row 32: 